# Labelling review

In [1]:
import os
import btrack
import napari
import glob
import zarr
import numpy as np

from macrohet import dataio, visualise
from tqdm.auto import tqdm

In [11]:
btrack.__version__

'0.6.1.dev127'

### Define functions

In [2]:
def track(objects, masks, config_fn, search_radius = 20):

    # initialise a tracker session using a context manager
    with btrack.BayesianTracker() as tracker:
        # configure the tracker using a config file
        tracker.configure(config_fn)
        # set max search radius
        tracker.max_search_radius = search_radius
        # define tracking method
        tracker.tracking_updates = ["MOTION", "VISUAL"]
        # redefine features so that both channels are included in track measurements
        tracker.features = list(objects[0].properties.keys())
        # append the objects to be tracked
        tracker.append(objects)
        # set the tracking volume
        tracker.volume=((0, masks.shape[-2]*scale_factor), (0, masks.shape[-1]*scale_factor))
        # track them (in interactive mode)
        tracker.track(step_size=25)
        # generate hypotheses and run the global optimizer
        tracker.optimize()
        # store the tracks
        tracks = tracker.tracks

    return tracks

In [3]:
# define tracking scale factor
scale_factor = 1/5.04

# define tracker config fn to use, using a prob_not_assign = 0.1
config_fn = '/home/dayn/analysis/models/btrack/particle_config_pnassign.json'

In [4]:
expt_ID = 'ND0002'
row, column = 3, 6
base_dir = f'/mnt/SYNO/macrohet_syno/{expt_ID}/'
acq_ID = (row, column)

In [6]:
os.path.join(base_dir, f'labels/cpv3/{row, column}_cpv3_mask_backup.h5')

'/mnt/SYNO/macrohet_syno/ND0002/labels/cpv3/(3, 6)_cpv3_mask_backup.h5'

In [ ]:
'/mnt/SYNO/macrohet_syno/ND0002/labels/cpv3/'

In [5]:
with btrack.io.HDF5FileHandler(os.path.join(base_dir, f'labels/cpv3/{row, column}_cpv3_mask_backup.h5'),
                                                   'r', 
                                                   obj_type='obj_type_1'
                                                   ) as reader:
                            masks = reader.segmentation
                 

[INFO][2024/04/02 11:50:25 AM] Opening HDF file: /mnt/SYNO/macrohet_syno/ND0002/labels/cpv3/(3, 6)_cpv3_mask_backup.h5...
INFO:btrack.io.hdf:Opening HDF file: /mnt/SYNO/macrohet_syno/ND0002/labels/cpv3/(3, 6)_cpv3_mask_backup.h5...
[INFO][2024/04/02 11:50:45 AM] Loading segmentation (150, 6048, 6048)
INFO:btrack.io.hdf:Loading segmentation (150, 6048, 6048)
[ERROR][2024/04/02 11:50:45 AM] Objects not found in /mnt/SYNO/macrohet_syno/ND0002/labels/cpv3/(3, 6)_cpv3_mask_backup.h5
ERROR:btrack.io.hdf:Objects not found in /mnt/SYNO/macrohet_syno/ND0002/labels/cpv3/(3, 6)_cpv3_mask_backup.h5
[ERROR][2024/04/02 11:50:45 AM] Tracks not found in /mnt/SYNO/macrohet_syno/ND0002/labels/cpv3/(3, 6)_cpv3_mask_backup.h5
ERROR:btrack.io.hdf:Tracks not found in /mnt/SYNO/macrohet_syno/ND0002/labels/cpv3/(3, 6)_cpv3_mask_backup.h5
[INFO][2024/04/02 11:50:45 AM] Closing HDF file: /mnt/SYNO/macrohet_syno/ND0002/labels/cpv3/(3, 6)_cpv3_mask_backup.h5
INFO:btrack.io.hdf:Closing HDF file: /mnt/SYNO/macrohet

In [7]:
with btrack.io.HDF5FileHandler(os.path.join(base_dir, f'labels/cpv3/{row, column}_cpv3_objects_backup.h5'),
                                                   'r', 
                                                   obj_type='obj_type_1'
                                                   ) as reader:
                            objects = reader.objects

[INFO][2024/04/02 11:52:20 AM] Opening HDF file: /mnt/SYNO/macrohet_syno/ND0002/labels/cpv3/(3, 6)_cpv3_objects_backup.h5...
INFO:btrack.io.hdf:Opening HDF file: /mnt/SYNO/macrohet_syno/ND0002/labels/cpv3/(3, 6)_cpv3_objects_backup.h5...
[INFO][2024/04/02 11:52:20 AM] Loading objects/obj_type_1 (32699, 5) (32699 filtered: None)
INFO:btrack.io.hdf:Loading objects/obj_type_1 (32699, 5) (32699 filtered: None)
[INFO][2024/04/02 11:52:20 AM] Closing HDF file: /mnt/SYNO/macrohet_syno/ND0002/labels/cpv3/(3, 6)_cpv3_objects_backup.h5
INFO:btrack.io.hdf:Closing HDF file: /mnt/SYNO/macrohet_syno/ND0002/labels/cpv3/(3, 6)_cpv3_objects_backup.h5


In [12]:
with btrack.io.HDF5FileHandler(os.path.join(base_dir, f'labels/cpv3/{row, column}_cpv3_track_backup.h5'),
                                                   'r', 
                                                   obj_type='obj_type_1'
                                                   ) as reader:
                            tracks = reader.tracks

[INFO][2024/04/02 11:53:12 AM] Opening HDF file: /mnt/SYNO/macrohet_syno/ND0002/labels/cpv3/(3, 6)_cpv3_track_backup.h5...
INFO:btrack.io.hdf:Opening HDF file: /mnt/SYNO/macrohet_syno/ND0002/labels/cpv3/(3, 6)_cpv3_track_backup.h5...


FileNotFoundError: [Errno 2] Unable to synchronously open file (unable to open file: name = '/mnt/SYNO/macrohet_syno/ND0002/labels/cpv3/(3, 6)_cpv3_track_backup.h5', errno = 2, error message = 'No such file or directory', flags = 0, o_flags = 0)

In [13]:
tracks = track(objects, masks, config_fn, search_radius = 20)



[INFO][2024/04/02 11:53:49 AM] Loaded btrack: /home/dayn/analysis/btrack/btrack/libs/libtracker.so
INFO:btrack.libwrapper:Loaded btrack: /home/dayn/analysis/btrack/btrack/libs/libtracker.so
[INFO][2024/04/02 11:53:49 AM] Starting BayesianTracker session
INFO:btrack.core:Starting BayesianTracker session
[INFO][2024/04/02 11:53:49 AM] Loading configuration file: /home/dayn/analysis/models/btrack/particle_config_pnassign.json
INFO:btrack.config:Loading configuration file: /home/dayn/analysis/models/btrack/particle_config_pnassign.json
[INFO][2024/04/02 11:53:49 AM] Objects are of type: <class 'list'>
INFO:btrack.io.utils:Objects are of type: <class 'list'>
[INFO][2024/04/02 11:53:50 AM] Starting tracking... 
INFO:btrack.core:Starting tracking... 
[INFO][2024/04/02 11:53:50 AM] Update using: ['MOTION', 'VISUAL']
INFO:btrack.core:Update using: ['MOTION', 'VISUAL']
[INFO][2024/04/02 11:53:50 AM] Tracking objects in frames 0 to 24 (of 150)...
INFO:btrack.core:Tracking objects in frames 0 to 2

GLPK Integer Optimizer 5.0
15736 rows, 12990 columns, 18112 non-zeros
12990 integer variables, all of which are binary
Preprocessing...
7868 rows, 12990 columns, 18112 non-zeros
12990 integer variables, all of which are binary
Scaling...
 A: min|aij| =  1.000e+00  max|aij| =  1.000e+00  ratio =  1.000e+00
Problem data seem to be well scaled
Constructing initial basis...
Size of triangular part is 7868
Solving LP relaxation...
GLPK Simplex Optimizer 5.0
7868 rows, 12990 columns, 18112 non-zeros
*     0: obj =   4.527371848e+04 inf =   0.000e+00 (3214)
Perturbing LP to avoid stalling [1704]...
Removing LP perturbation [3241]...
*  3241: obj =   2.349576664e+04 inf =   0.000e+00 (0)
OPTIMAL LP SOLUTION FOUND
Integer optimization begins...
Long-step dual simplex will be used
+  3241: mip =     not found yet >=              -inf        (1; 0)
+  3241: >>>>>   2.349576664e+04 >=   2.349576664e+04   0.0% (1; 0)
+  3241: mip =   2.349576664e+04 >=     tree is empty   0.0% (0; 1)
INTEGER OPTIMA

[INFO][2024/04/02 11:53:59 AM] Ending BayesianTracker session
INFO:btrack.core:Ending BayesianTracker session


In [15]:
with btrack.io.HDF5FileHandler('test_tracks.h5',
                               'w', 
                               obj_type='obj_type_1'
                                                   ) as writer:
                            writer.write_tracks(tracks)

[INFO][2024/04/02 11:55:21 AM] Opening HDF file: test_tracks.h5...
INFO:btrack.io.hdf:Opening HDF file: test_tracks.h5...
[INFO][2024/04/02 11:55:22 AM] Writing objects/obj_type_1
INFO:btrack.io.hdf:Writing objects/obj_type_1
[INFO][2024/04/02 11:55:22 AM] Writing labels/obj_type_1
INFO:btrack.io.hdf:Writing labels/obj_type_1
[INFO][2024/04/02 11:55:22 AM] Loading objects/obj_type_1 (27965, 5) (27965 filtered: None)
INFO:btrack.io.hdf:Loading objects/obj_type_1 (27965, 5) (27965 filtered: None)
[INFO][2024/04/02 11:55:22 AM] Writing properties/obj_type_1/major_axis_length (27965,)
INFO:btrack.io.hdf:Writing properties/obj_type_1/major_axis_length (27965,)
[INFO][2024/04/02 11:55:22 AM] Writing properties/obj_type_1/orientation (27965,)
INFO:btrack.io.hdf:Writing properties/obj_type_1/orientation (27965,)
[INFO][2024/04/02 11:55:22 AM] Writing properties/obj_type_1/minor_axis_length (27965,)
INFO:btrack.io.hdf:Writing properties/obj_type_1/minor_axis_length (27965,)
[INFO][2024/04/02 11

In [16]:
with btrack.io.HDF5FileHandler('test_tracks.h5',
                               'r', 
                               obj_type='obj_type_1'
                               ) as reader:
                            test_input = reader.tracks

[INFO][2024/04/02 11:56:07 AM] Opening HDF file: test_tracks.h5...
INFO:btrack.io.hdf:Opening HDF file: test_tracks.h5...
[INFO][2024/04/02 11:56:07 AM] Loading tracks/obj_type_1
INFO:btrack.io.hdf:Loading tracks/obj_type_1
[INFO][2024/04/02 11:56:07 AM] Loading LBEP/obj_type_1
INFO:btrack.io.hdf:Loading LBEP/obj_type_1
[INFO][2024/04/02 11:56:07 AM] Loading objects/obj_type_1 (27965, 5) (27965 filtered: None)
INFO:btrack.io.hdf:Loading objects/obj_type_1 (27965, 5) (27965 filtered: None)
[INFO][2024/04/02 11:56:08 AM] Closing HDF file: test_tracks.h5
INFO:btrack.io.hdf:Closing HDF file: test_tracks.h5


In [17]:
test_input[0]

,ID,t,x,y,z,parent,root,state,generation,dummy,major_axis_length,Infected,mean_intensity,orientation,minor_axis_length,area,Mtb area px
0,166,0,23.761169,532.807800,0.000000,166,166,5,0,False,315.532318,1.0,"(3,) array",0.231978,126.334122,30282.0,1401.0
1,166,1,23.508297,534.426636,0.000000,166,166,5,0,False,301.344696,1.0,"(3,) array",0.151356,122.023491,27224.0,1366.0
2,166,2,24.184298,534.813721,0.000000,166,166,5,0,False,295.125519,1.0,"(3,) array",0.212196,129.490326,28784.0,1763.0
3,166,3,23.841808,525.442444,0.000000,166,166,5,0,False,441.298889,1.0,"(3,) array",0.036206,120.516502,33519.0,1679.0
4,166,4,24.783676,527.587097,0.000000,166,166,5,0,False,368.665131,1.0,"(3,) array",0.023249,129.842773,33300.0,1983.0
5,166,5,24.865301,529.760315,0.000000,166,166,5,0,False,312.658356,1.0,"(3,) array",0.042914,126.721672,29276.0,1945.0
6,166,6,26.168837,532.338928,0.000000,166,166,5,0,False,308.008179,1.0,"(3,) array",0.098562,126.584793,28911.0,2056.0
7,166,7,26.230917,532.017395,0.000000,166,166,5,0,False,317.969421,1.0,"(3,) array",0.114913,126.061172,30203.0,1947.0
8,166,8,26.354191,534.261292,0.000000,166,166,5,0,False,296.316864,1.0,"(3,) array",0.196343,130.042801,29152.0,2010.0
9,166,9,26.658426,535.006775,0.000000,166,166,5,0,False,370.018219,1.0,"(3,) array",0.053670,128.859344,34115.0,2004.0


In [18]:
with btrack.io.HDF5FileHandler(os.path.join(base_dir, f'labels/cpv3/{row, column}_cpv3_tracks.h5'),
                       'w', 
                       obj_type='obj_type_1'
                                           ) as writer:
                    writer.write_tracks(tracks)
                    writer.write_segmentation(masks)

[INFO][2024/04/02 12:01:15 PM] Opening HDF file: /mnt/SYNO/macrohet_syno/ND0002/labels/cpv3/(3, 6)_cpv3_tracks.h5...
INFO:btrack.io.hdf:Opening HDF file: /mnt/SYNO/macrohet_syno/ND0002/labels/cpv3/(3, 6)_cpv3_tracks.h5...
[INFO][2024/04/02 12:01:15 PM] Writing objects/obj_type_1
INFO:btrack.io.hdf:Writing objects/obj_type_1
[INFO][2024/04/02 12:01:15 PM] Writing labels/obj_type_1
INFO:btrack.io.hdf:Writing labels/obj_type_1
[INFO][2024/04/02 12:01:15 PM] Loading objects/obj_type_1 (27965, 5) (27965 filtered: None)
INFO:btrack.io.hdf:Loading objects/obj_type_1 (27965, 5) (27965 filtered: None)
[INFO][2024/04/02 12:01:15 PM] Writing properties/obj_type_1/major_axis_length (27965,)
INFO:btrack.io.hdf:Writing properties/obj_type_1/major_axis_length (27965,)
[INFO][2024/04/02 12:01:15 PM] Writing properties/obj_type_1/orientation (27965,)
INFO:btrack.io.hdf:Writing properties/obj_type_1/orientation (27965,)
[INFO][2024/04/02 12:01:15 PM] Writing properties/obj_type_1/minor_axis_length (2796

In [24]:
with btrack.io.HDF5FileHandler(os.path.join(base_dir, f'labels/cpv3/{row, column}_cpv3_tracks.h5'),
                                                   'r', 
                                                   obj_type='obj_type_1'
                                                   ) as reader:
                            # test_tracks = reader.tracks
                            # seg = reader.segmentation
                            objs = reader.objects

[INFO][2024/04/02 12:06:13 PM] Opening HDF file: /mnt/SYNO/macrohet_syno/ND0002/labels/cpv3/(3, 6)_cpv3_tracks.h5...
INFO:btrack.io.hdf:Opening HDF file: /mnt/SYNO/macrohet_syno/ND0002/labels/cpv3/(3, 6)_cpv3_tracks.h5...
[INFO][2024/04/02 12:06:13 PM] Loading objects/obj_type_1 (27965, 5) (27965 filtered: None)
INFO:btrack.io.hdf:Loading objects/obj_type_1 (27965, 5) (27965 filtered: None)
[INFO][2024/04/02 12:06:14 PM] Closing HDF file: /mnt/SYNO/macrohet_syno/ND0002/labels/cpv3/(3, 6)_cpv3_tracks.h5
INFO:btrack.io.hdf:Closing HDF file: /mnt/SYNO/macrohet_syno/ND0002/labels/cpv3/(3, 6)_cpv3_tracks.h5


In [20]:
test_tracks[0]

,ID,t,x,y,z,parent,root,state,generation,dummy,major_axis_length,Infected,mean_intensity,orientation,minor_axis_length,area,Mtb area px
0,166,0,23.761169,532.807800,0.000000,166,166,5,0,False,315.532318,1.0,"(3,) array",0.231978,126.334122,30282.0,1401.0
1,166,1,23.508297,534.426636,0.000000,166,166,5,0,False,301.344696,1.0,"(3,) array",0.151356,122.023491,27224.0,1366.0
2,166,2,24.184298,534.813721,0.000000,166,166,5,0,False,295.125519,1.0,"(3,) array",0.212196,129.490326,28784.0,1763.0
3,166,3,23.841808,525.442444,0.000000,166,166,5,0,False,441.298889,1.0,"(3,) array",0.036206,120.516502,33519.0,1679.0
4,166,4,24.783676,527.587097,0.000000,166,166,5,0,False,368.665131,1.0,"(3,) array",0.023249,129.842773,33300.0,1983.0
5,166,5,24.865301,529.760315,0.000000,166,166,5,0,False,312.658356,1.0,"(3,) array",0.042914,126.721672,29276.0,1945.0
6,166,6,26.168837,532.338928,0.000000,166,166,5,0,False,308.008179,1.0,"(3,) array",0.098562,126.584793,28911.0,2056.0
7,166,7,26.230917,532.017395,0.000000,166,166,5,0,False,317.969421,1.0,"(3,) array",0.114913,126.061172,30203.0,1947.0
8,166,8,26.354191,534.261292,0.000000,166,166,5,0,False,296.316864,1.0,"(3,) array",0.196343,130.042801,29152.0,2010.0
9,166,9,26.658426,535.006775,0.000000,166,166,5,0,False,370.018219,1.0,"(3,) array",0.053670,128.859344,34115.0,2004.0


In [ ]:
for expt_ID in expts_to_review:

    
    metadata_fn = os.path.join(base_dir, 'acquisition/Images/Index.idx.xml')
    metadata = dataio.read_harmony_metadata(metadata_fn)  
    metadata_path = glob.glob(os.path.join(base_dir, 'acquisition/Assaylayout/*.xml'))[0]
    assay_layout = dataio.read_harmony_metadata(metadata_path, assay_layout=True,)
    
    for (row, column), info in tqdm(assay_layout.iterrows(), desc='Progress through positions', total=len(assay_layout), leave = False):
    
        acq_ID = (row, column)
    
        # process images using zarr
        image_dir = os.path.join(base_dir, f'acquisition/zarr/{acq_ID}.zarr')
        zarr_store = zarr.open(image_dir, mode='r')
        zarr_images = zarr_store.images
        
        # create a max projection
        # %time images = np.max(zarr_images, axis = 2)
        # for times sake only load z0
        images_z0 = zarr_images[:,:,0,...]
        # load objects and segmentation 
        with btrack.io.HDF5FileHandler(os.path.join(base_dir, f'labels/cpv3/{row, column}_cpv3_mask_backup.h5'),
                                                   'r', 
                                                   obj_type='obj_type_1'
                                                   ) as reader:
                            masks = reader.segmentation

        if os.path.exists(f'{expt_ID}.{row}.{column}_filtered_napari_tracks.npy'):
            print('Loading tracks')
            napari_tracks = np.load(f'{expt_ID}.{row}.{column}_filtered_napari_tracks.npy')
            
        else:
            print('Tracking tracks')
            with btrack.io.HDF5FileHandler(os.path.join(base_dir, f'labels/cpv3/{row, column}_cpv3_objects_backup.h5'),
                                                       'r', 
                                                       obj_type='obj_type_1'
                                                       ) as reader:
                                objects = reader.objects
    
            # retrack as fuggin tracks didn't save out
            tracks = track(objects, masks, config_fn, search_radius = 20)
    
            # apply length filter for tracks
            filtered_tracks = [t for t in tracks if len(t) >= 70]
    
            # convert to napari format
            napari_tracks, _, _ = btrack.utils.tracks_to_napari(filtered_tracks, ndim = 2)
            np.save(f'{expt_ID}.{row}.{column}_filtered_napari_tracks.npy', napari_tracks)
            np.save(f'{expt_ID}.{row}.{column}_napari_tracks.npy', btrack.utils.tracks_to_napari(tracks, ndim = 2))

        # launch  napari viewer
        viewer = napari.Viewer(title = f'{expt_ID}, {row, column}')
        viewer.add_image(images_z0, channel_axis=1, contrast_limits=[[0,1000], [0, 2000]])
        viewer.add_labels(masks)
        viewer.add_tracks(napari_tracks, scale = (1/scale_factor, 1/scale_factor))
        viewer_tools(viewer)
        visualise.add_napari_grid_overlay(viewer)
        viewer.show(block=True)

Reading metadata XML file...


0it [00:00, ?it/s]

Extracting metadata complete!
Reading metadata XML file...
Extracting metadata complete!


Progress through positions:   0%|          | 0/42 [00:00<?, ?it/s]

[INFO][2024/03/27 11:56:10 AM] Opening HDF file: /mnt/SYNO/macrohet_syno/ND0002/labels/cpv3/(3, 1)_cpv3_mask_backup.h5...
INFO:btrack.io.hdf:Opening HDF file: /mnt/SYNO/macrohet_syno/ND0002/labels/cpv3/(3, 1)_cpv3_mask_backup.h5...
[INFO][2024/03/27 11:56:31 AM] Loading segmentation (150, 6048, 6048)
INFO:btrack.io.hdf:Loading segmentation (150, 6048, 6048)
[INFO][2024/03/27 11:56:31 AM] Closing HDF file: /mnt/SYNO/macrohet_syno/ND0002/labels/cpv3/(3, 1)_cpv3_mask_backup.h5
INFO:btrack.io.hdf:Closing HDF file: /mnt/SYNO/macrohet_syno/ND0002/labels/cpv3/(3, 1)_cpv3_mask_backup.h5


...

Loading tracks


[INFO][2024/03/27 01:44:29 pm] Opening HDF file: /mnt/SYNO/macrohet_syno/ND0002/labels/cpv3/(3, 5)_cpv3_mask_backup.h5...
INFO:btrack.io.hdf:Opening HDF file: /mnt/SYNO/macrohet_syno/ND0002/labels/cpv3/(3, 5)_cpv3_mask_backup.h5...
[INFO][2024/03/27 01:45:11 pm] Loading segmentation (150, 6048, 6048)
INFO:btrack.io.hdf:Loading segmentation (150, 6048, 6048)
[INFO][2024/03/27 01:45:11 pm] Closing HDF file: /mnt/SYNO/macrohet_syno/ND0002/labels/cpv3/(3, 5)_cpv3_mask_backup.h5
INFO:btrack.io.hdf:Closing HDF file: /mnt/SYNO/macrohet_syno/ND0002/labels/cpv3/(3, 5)_cpv3_mask_backup.h5


Loading tracks


[INFO][2024/03/27 04:36:25 pm] Opening HDF file: /mnt/SYNO/macrohet_syno/ND0002/labels/cpv3/(3, 6)_cpv3_mask_backup.h5...
INFO:btrack.io.hdf:Opening HDF file: /mnt/SYNO/macrohet_syno/ND0002/labels/cpv3/(3, 6)_cpv3_mask_backup.h5...
[INFO][2024/03/27 04:37:16 pm] Loading segmentation (150, 6048, 6048)
INFO:btrack.io.hdf:Loading segmentation (150, 6048, 6048)
[INFO][2024/03/27 04:37:16 pm] Closing HDF file: /mnt/SYNO/macrohet_syno/ND0002/labels/cpv3/(3, 6)_cpv3_mask_backup.h5
INFO:btrack.io.hdf:Closing HDF file: /mnt/SYNO/macrohet_syno/ND0002/labels/cpv3/(3, 6)_cpv3_mask_backup.h5


Loading tracks


In [ ]:
break

# Track all pos and save to dict 

In [30]:
for expt_ID in ['ND0002', 'ND0003']:

    base_dir = f'/mnt/SYNO/macrohet_syno/{expt_ID}/'
    metadata_fn = os.path.join(base_dir, 'acquisition/Images/Index.idx.xml')
    metadata = dataio.read_harmony_metadata(metadata_fn)  
    metadata_path = glob.glob(os.path.join(base_dir, 'acquisition/Assaylayout/*.xml'))[0]
    assay_layout = dataio.read_harmony_metadata(metadata_path, assay_layout=True,)
    
    for (row, column), info in tqdm(assay_layout.iterrows(), desc='Progress through positions', total=len(assay_layout), leave = False):
        if os.path.exists(os.path.join(base_dir, f'labels/cpv3/{row, column}_cpv3_tracks.h5')):
            continue
        try:
            # load objects and segmentation 
            with btrack.io.HDF5FileHandler(os.path.join(base_dir, f'labels/cpv3/backup/{row, column}_cpv3_mask_backup.h5'),
                                                       'r', 
                                                       obj_type='obj_type_1'
                                                       ) as reader:
                                masks = reader.segmentation
            with btrack.io.HDF5FileHandler(os.path.join(base_dir, f'labels/cpv3/backup/{row, column}_cpv3_objects_backup.h5'),
                                                       'r', 
                                                       obj_type='obj_type_1'
                                                       ) as reader:
                                objects = reader.objects
    
            # retrack as fuggin tracks didn't save out
            tracks = track(objects, masks, config_fn, search_radius = 20)
    
            
            with btrack.io.HDF5FileHandler(os.path.join(base_dir, f'labels/cpv3/{row, column}_cpv3_tracks.h5'),
                                   'w', 
                                   obj_type='obj_type_1'
                                                       ) as writer:
                                writer.write_tracks(tracks)
                                writer.write_segmentation(masks)
        except:
            print(os.path.join(base_dir, f'labels/cpv3/{row, column}_cpv3_tracks.h5'))

Reading metadata XML file...


0it [00:00, ?it/s]

Extracting metadata complete!
Reading metadata XML file...
Extracting metadata complete!


Progress through positions:   0%|          | 0/42 [00:00<?, ?it/s]

[INFO][2024/04/02 01:42:39 PM] Opening HDF file: /mnt/SYNO/macrohet_syno/ND0002/labels/cpv3/backup/(6, 3)_cpv3_mask_backup.h5...
INFO:btrack.io.hdf:Opening HDF file: /mnt/SYNO/macrohet_syno/ND0002/labels/cpv3/backup/(6, 3)_cpv3_mask_backup.h5...


...

[INFO][2024/04/02 02:40:40 PM] Ending BayesianTracker session
INFO:btrack.core:Ending BayesianTracker session
[INFO][2024/04/02 02:40:41 PM] Opening HDF file: /mnt/SYNO/macrohet_syno/ND0003/labels/cpv3/(6, 10)_cpv3_tracks.h5...
INFO:btrack.io.hdf:Opening HDF file: /mnt/SYNO/macrohet_syno/ND0003/labels/cpv3/(6, 10)_cpv3_tracks.h5...
[INFO][2024/04/02 02:40:41 PM] Writing objects/obj_type_1
INFO:btrack.io.hdf:Writing objects/obj_type_1
[INFO][2024/04/02 02:40:41 PM] Writing labels/obj_type_1
INFO:btrack.io.hdf:Writing labels/obj_type_1
[INFO][2024/04/02 02:40:41 PM] Loading objects/obj_type_1 (34869, 5) (34869 filtered: None)
INFO:btrack.io.hdf:Loading objects/obj_type_1 (34869, 5) (34869 filtered: None)
[INFO][2024/04/02 02:40:41 PM] Writing properties/obj_type_1/major_axis_length (34869,)
INFO:btrack.io.hdf:Writing properties/obj_type_1/major_axis_length (34869,)
[INFO][2024/04/02 02:40:41 PM] Writing properties/obj_type_1/orientation (34869,)
INFO:btrack.io.hdf:Writing properties/obj_

GLPK Integer Optimizer 5.0
13716 rows, 11446 columns, 16034 non-zeros
11446 integer variables, all of which are binary
Preprocessing...
6858 rows, 11446 columns, 16034 non-zeros
11446 integer variables, all of which are binary
Scaling...
 A: min|aij| =  1.000e+00  max|aij| =  1.000e+00  ratio =  1.000e+00
Problem data seem to be well scaled
Constructing initial basis...
Size of triangular part is 6858
Solving LP relaxation...
GLPK Simplex Optimizer 5.0
6858 rows, 11446 columns, 16034 non-zeros
*     0: obj =   3.987344179e+04 inf =   0.000e+00 (3198)
Perturbing LP to avoid stalling [1561]...
Removing LP perturbation [3250]...
*  3250: obj =   1.905349124e+04 inf =   0.000e+00 (0)
OPTIMAL LP SOLUTION FOUND
Integer optimization begins...
Long-step dual simplex will be used
+  3250: mip =     not found yet >=              -inf        (1; 0)
+  3250: >>>>>   1.905349124e+04 >=   1.905349124e+04   0.0% (1; 0)
+  3250: mip =   1.905349124e+04 >=     tree is empty   0.0% (0; 1)
INTEGER OPTIMA

[INFO][2024/04/02 02:42:04 PM] Ending BayesianTracker session
INFO:btrack.core:Ending BayesianTracker session
[INFO][2024/04/02 02:42:04 PM] Opening HDF file: /mnt/SYNO/macrohet_syno/ND0003/labels/cpv3/(6, 11)_cpv3_tracks.h5...
INFO:btrack.io.hdf:Opening HDF file: /mnt/SYNO/macrohet_syno/ND0003/labels/cpv3/(6, 11)_cpv3_tracks.h5...
[INFO][2024/04/02 02:42:05 PM] Writing objects/obj_type_1
INFO:btrack.io.hdf:Writing objects/obj_type_1
[INFO][2024/04/02 02:42:05 PM] Writing labels/obj_type_1
INFO:btrack.io.hdf:Writing labels/obj_type_1
[INFO][2024/04/02 02:42:05 PM] Loading objects/obj_type_1 (24607, 5) (24607 filtered: None)
INFO:btrack.io.hdf:Loading objects/obj_type_1 (24607, 5) (24607 filtered: None)
[INFO][2024/04/02 02:42:05 PM] Writing properties/obj_type_1/major_axis_length (24607,)
INFO:btrack.io.hdf:Writing properties/obj_type_1/major_axis_length (24607,)
[INFO][2024/04/02 02:42:05 PM] Writing properties/obj_type_1/orientation (24607,)
INFO:btrack.io.hdf:Writing properties/obj_

GLPK Integer Optimizer 5.0
8840 rows, 7193 columns, 9966 non-zeros
7193 integer variables, all of which are binary
Preprocessing...
4420 rows, 7193 columns, 9966 non-zeros
7193 integer variables, all of which are binary
Scaling...
 A: min|aij| =  1.000e+00  max|aij| =  1.000e+00  ratio =  1.000e+00
Problem data seem to be well scaled
Constructing initial basis...
Size of triangular part is 4420
Solving LP relaxation...
GLPK Simplex Optimizer 5.0
4420 rows, 7193 columns, 9966 non-zeros
*     0: obj =   2.541857007e+04 inf =   0.000e+00 (1835)
Perturbing LP to avoid stalling [1102]...
Removing LP perturbation [1852]...
*  1852: obj =   1.302340672e+04 inf =   0.000e+00 (0)
OPTIMAL LP SOLUTION FOUND
Integer optimization begins...
Long-step dual simplex will be used
+  1852: mip =     not found yet >=              -inf        (1; 0)
+  1852: >>>>>   1.302340672e+04 >=   1.302340672e+04   0.0% (1; 0)
+  1852: mip =   1.302340672e+04 >=     tree is empty   0.0% (0; 1)
INTEGER OPTIMAL SOLUTIO

[INFO][2024/04/02 02:43:22 PM] Opening HDF file: /mnt/SYNO/macrohet_syno/ND0003/labels/cpv3/(6, 12)_cpv3_tracks.h5...
INFO:btrack.io.hdf:Opening HDF file: /mnt/SYNO/macrohet_syno/ND0003/labels/cpv3/(6, 12)_cpv3_tracks.h5...
[INFO][2024/04/02 02:43:22 PM] Writing objects/obj_type_1
INFO:btrack.io.hdf:Writing objects/obj_type_1
[INFO][2024/04/02 02:43:22 PM] Writing labels/obj_type_1
INFO:btrack.io.hdf:Writing labels/obj_type_1
[INFO][2024/04/02 02:43:22 PM] Loading objects/obj_type_1 (17246, 5) (17246 filtered: None)
INFO:btrack.io.hdf:Loading objects/obj_type_1 (17246, 5) (17246 filtered: None)
[INFO][2024/04/02 02:43:22 PM] Writing properties/obj_type_1/major_axis_length (17246,)
INFO:btrack.io.hdf:Writing properties/obj_type_1/major_axis_length (17246,)
[INFO][2024/04/02 02:43:22 PM] Writing properties/obj_type_1/orientation (17246,)
INFO:btrack.io.hdf:Writing properties/obj_type_1/orientation (17246,)
[INFO][2024/04/02 02:43:22 PM] Writing properties/obj_type_1/minor_axis_length (17